In [1]:
# %pip install plotly
# %pip install nbformat
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

os.getcwd()

'/Users/aneeshussain/Code/diss/analysis'

In [2]:
# Get the current directory
current_dir = os.getcwd()

# Change directory to the parent directory (diss)
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
os.chdir(parent_dir)

# Append the parent directory to the system path (optional, if you want to add it to the import path)
sys.path.append(parent_dir)

# Print the new current working directory to confirm the change
print(f"Changed directory to: {os.getcwd()}")

Changed directory to: /Users/aneeshussain/Code/diss


In [3]:
import pandas as pd

# Filepaths for each exchange
filepaths = {
    'binance': './data/processed/binance.csv',
    'okx': './data/processed/okx.csv',
    'bybit': './data/processed/bybit.csv'
}

# List of specified pairs
specified_pairs = ['BTCUSDCM', 'BTCUSDT', 'ETHUSDCM', 'ETHUSDT']

# Initialize a dictionary to store counts for each exchange
exchange_funding_stats = {}

# Loop through each exchange
for exchange, filepath in filepaths.items():
    # Load the data for the current exchange
    df = pd.read_csv(filepath)

    # Convert 'funding rate' to float
    df['funding rate'] = pd.to_numeric(df['funding rate'], errors='coerce')

    # Filter the DataFrame to only include the specified pairs
    filtered_df = df[df['pair'].isin(specified_pairs)]

    # Initialize dictionaries to store counts for this exchange
    positive_funding_counts = {}
    total_funding_counts = {}

    # Iterate through each specified pair
    for pair in specified_pairs:
        # Filter rows for the current pair
        pair_df = filtered_df[filtered_df['pair'] == pair]

        # Count positive funding rates
        positive_count = (pair_df['funding rate'] > 0).sum()
        total_count = len(pair_df)

        # Store the counts in the dictionaries
        positive_funding_counts[pair] = positive_count
        total_funding_counts[pair] = total_count

    # Create a DataFrame for this exchange to display the counts
    funding_stats = pd.DataFrame({
        'positive_count': positive_funding_counts,
        'total_count': total_funding_counts
    })

    # Calculate the percentage of positive funding rates
    funding_stats['positive_percentage'] = (funding_stats['positive_count'] / funding_stats['total_count']) * 100

    # Store the result for this exchange
    exchange_funding_stats[exchange] = funding_stats

# Display the results for each exchange
for exchange, stats in exchange_funding_stats.items():
    print(f"{exchange} Funding stats")
    print(stats)
    print("\n")

binance Funding stats
          positive_count  total_count  positive_percentage
BTCUSDCM            1435         1642            87.393423
BTCUSDT             1517         1642            92.387333
ETHUSDCM            1467         1642            89.342266
ETHUSDT             1539         1642            93.727162


okx Funding stats
          positive_count  total_count  positive_percentage
BTCUSDCM            1132         1642            68.940317
BTCUSDT             1203         1642            73.264312
ETHUSDCM            1163         1642            70.828258
ETHUSDT             1226         1642            74.665043


bybit Funding stats
          positive_count  total_count  positive_percentage
BTCUSDCM            1472         1642            89.646772
BTCUSDT             1494         1642            90.986602
ETHUSDCM            1481         1642            90.194884
ETHUSDT             1474         1642            89.768575




In [80]:
trades_filepath = './results/simple_threshold_open/trades_log.csv'
trades_df = pd.read_csv(trades_filepath)

# Convert 'open_time' and 'close_time' to datetime
trades_df['open_time'] = pd.to_datetime(trades_df['open_time'])
trades_df['close_time'] = pd.to_datetime(trades_df['close_time'])

# Filter for long positions
long_trades_df = trades_df[trades_df['position_type'] == 'long']

exchange_trade_count = long_trades_df['exchange'].value_counts()

# Display the count of trades on each exchange
print(exchange_trade_count)

exchange
okx        1128
bybit       353
binance     344
Name: count, dtype: int64


In [140]:
import pandas as pd
import plotly.graph_objects as go

yield_filepath = './results/test/yield.csv'
yield_df = pd.read_csv(yield_filepath)

yield_df['btc_yield'] = yield_df['binance_btc_yield'] + yield_df['okx_btc_yield'] + yield_df['bybit_btc_yield']
yield_df['eth_yield'] = yield_df['binance_eth_yield'] + yield_df['okx_eth_yield'] + yield_df['bybit_eth_yield']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['btc_yield'],
    mode='lines',
    name='Avg BTC Yield',
    line=dict(color='orange', width=2),
    opacity=0.8
))

fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['eth_yield'],
    mode='lines',
    name='Avg ETH Yield',
    line=dict(color='green', width=1.5),
    opacity=0.8
))

fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['total_yield'],
    mode='lines',
    name='Avg Perp Yield',
    line=dict(color='blue', width=1.25),
    opacity=0.8
))

fig.update_layout(
    title='Volume Weighted Average Perpetual Yield',
    yaxis_title='Average Yield (%)',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation='h',
        x=0.5,
        y=-0.2,
        xanchor='center',
        yanchor='top'
    ),
    width=800,
    height=600,
    margin=dict(l=40, r=40, t=80, b=40)
)

fig.update_xaxes(
    tickformat='%b %Y',
)

fig.show()

In [151]:
import pandas as pd
import plotly.graph_objects as go

yield_filepath = './results/test/yield.csv'
yield_df = pd.read_csv(yield_filepath)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['bybit_btccm_yield'] / 0.025,
    mode='lines',
    name='Bybit BTCUSD',
    line=dict(color='red', width=2),
    opacity=0.7,
    legendrank=5
))

fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['okx_btccm_yield'] / 0.0225,
    mode='lines',
    name='OKX BTC-USD-SWAP',
    line=dict(color='cyan', width=2),
    opacity=0.7,
    legendrank=6
))

fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['binance_btccm_yield'] / 0.09,
    mode='lines',
    name='Binance BTCUSD_PERP',
    line=dict(color='orange', width=2),
    opacity=0.7,
    legendrank=4
))


fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['binance_btcusd_yield'] / 0.36,
    mode='lines',
    name='Binance BTC/USDT:USDT',
    line=dict(color='blue', width=2),
    opacity=0.7,
    legendrank=1
))


fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['bybit_btcusd_yield'] / 0.11,
    mode='lines',
    name='Bybit BTC/USDT:USDT',
    line=dict(color='green', width=2),
    opacity=0.7,
    legendrank=2
))


fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['okx_btcusd_yield'] / 0.09,
    mode='lines',
    name='OKX BTC-USDT-SWAP',
    line=dict(color='purple', width=2),
    opacity=0.7,
    legendrank=3
))

# fig.add_trace(go.Scatter(
#     x=yield_df['date'],
#     y=yield_df['bybit_btccm_yield'],
#     mode='lines',
#     name='Bybit BTCUSD',
#     line=dict(color='red', width=2),
#     opacity=0.7,
#     legendrank=5
# ))

# fig.add_trace(go.Scatter(
#     x=yield_df['date'],
#     y=yield_df['okx_btccm_yield'],
#     mode='lines',
#     name='OKX BTC-USD-SWAP',
#     line=dict(color='cyan', width=2),
#     opacity=0.7,
#     legendrank=6
# ))

# fig.add_trace(go.Scatter(
#     x=yield_df['date'],
#     y=yield_df['binance_btccm_yield'],
#     mode='lines',
#     name='Binance BTCUSD_PERP',
#     line=dict(color='orange', width=2),
#     opacity=0.7,
#     legendrank=4
# ))


# fig.add_trace(go.Scatter(
#     x=yield_df['date'],
#     y=yield_df['binance_btcusd_yield'],
#     mode='lines',
#     name='Binance BTC/USDT:USDT',
#     line=dict(color='blue', width=2),
#     opacity=0.7,
#     legendrank=1
# ))


# fig.add_trace(go.Scatter(
#     x=yield_df['date'],
#     y=yield_df['bybit_btcusd_yield'],
#     mode='lines',
#     name='Bybit BTC/USDT:USDT',
#     line=dict(color='green', width=2),
#     opacity=0.7,
#     legendrank=2
# ))


# fig.add_trace(go.Scatter(
#     x=yield_df['date'],
#     y=yield_df['okx_btcusd_yield'],
#     mode='lines',
#     name='OKX BTC-USDT-SWAP',
#     line=dict(color='purple', width=2),
#     opacity=0.7,
#     legendrank=3
# ))

fig.update_layout(
    title='Historic BTC Perpetual Yields',
    yaxis_title='Yield (%)',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation='h',
        x=0.5,
        y=-0.2,
        xanchor='center',
        yanchor='top'
    ),
    width=800,
    height=600,
    margin=dict(l=40, r=40, t=80, b=40)
)

fig.update_xaxes(
    tickformat='%b %Y',
)

fig.show()

In [148]:
import pandas as pd
import plotly.graph_objects as go

yield_filepath = './results/test/yield.csv'
yield_df = pd.read_csv(yield_filepath)

total_yields = {
    'Binance BTC/USDT:USDT': (yield_df['binance_btcusd_yield']).sum(), #  / 0.36
    'Binance BTCUSD_PERP': (yield_df['binance_btccm_yield']).sum(), #  / 0.09
    'Bybit BTC/USDT:USDT': (yield_df['bybit_btcusd_yield']).sum(), #  / 0.11
    'Bybit BTCUSD': (yield_df['bybit_btccm_yield']).sum(),  #  / 0.025
    'OKX BTC-USDT-SWAP': (yield_df['okx_btcusd_yield']).sum(),   #  / 0.09
    'OKX BTC-USD-SWAP': (yield_df['okx_btccm_yield']).sum(),   #  / 0.0225
}

total_yields_df = pd.DataFrame(list(total_yields.items()), columns=['Asset', 'Cumulative Yield'])

fig = go.Figure()

fig.add_trace(go.Bar(
    x=total_yields_df['Asset'],
    y=total_yields_df['Cumulative Yield'],
    marker_color=['blue', 'orange', 'green', 'red', 'purple', 'cyan'],  
    opacity=1.0
))

fig.update_layout(
    title='Cumulative BTC Perpetual Yields',
    # xaxis_title='Asset',
    yaxis_title='Cumulative Yield (%)',
    template='plotly_white',
    hovermode='x unified',
    width=800,
    height=600,
    margin=dict(l=40, r=40, t=80, b=40)
)

fig.show()

In [155]:
import pandas as pd
import plotly.graph_objects as go

yield_filepath = './results/test/yield.csv'
yield_df = pd.read_csv(yield_filepath)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['binance_btc_yield'] / 0.54,
    mode='lines',
    name='Binance',
    line=dict(color='blue', width=2),
    opacity=0.7,
    legendrank=1
))

fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['bybit_btc_yield'] / 0.11,
    mode='lines',
    name='Bybit',
    line=dict(color='orange', width=2),
    opacity=0.7,
    legendrank=2
))


fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['okx_btc_yield'] / 0.09,
    mode='lines',
    name='OKX',
    line=dict(color='green', width=2),
    opacity=0.7,
    legendrank=3
))

fig.update_layout(
    title='BTC Exchange Perpetual Yields',
    yaxis_title='Yield (%)',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation='h',
        x=0.5,
        y=-0.2,
        xanchor='center',
        yanchor='top'
    ),
    width=800,
    height=600,
    margin=dict(l=40, r=40, t=80, b=40)
)

fig.update_xaxes(
    tickformat='%b %Y',
)

fig.show()

In [144]:
import pandas as pd
import plotly.graph_objects as go

yield_filepath = './results/test/yield.csv'
yield_df = pd.read_csv(yield_filepath)

btc_columns = [
    'binance_btcusd_yield',
    'binance_btccm_yield',
    'bybit_btcusd_yield',
    'bybit_btccm_yield',
    'okx_btcusd_yield',
    'okx_btccm_yield'
]

# Narrow down to just the specified BTC asset columns
btc_yield_df = yield_df[btc_columns]

positive_counts = (btc_yield_df >= 0).sum().sum()
negative_counts = (btc_yield_df < 0).sum().sum()


fig = go.Figure()


fig.add_trace(go.Pie(
    values=[positive_counts, negative_counts], 
    labels=['Positive', 'Negative'],
    textinfo='percent',
    hole=0.7, 
    sort=False, 
    direction='counterclockwise', 
    marker=dict(colors=['green', 'red']),
    opacity=1.0
))


fig.update_layout(
    title_text='BTC Perpetual Yield Day Counts',
    showlegend=True,
    legend=dict(
        orientation='h',
        x=0.5,
        y=-0.2,
        xanchor='center',
        yanchor='top'
    ),
    height=300,
    width=300
)

fig.show()

In [145]:
import pandas as pd
import plotly.graph_objects as go

yield_filepath = './results/test/yield.csv'
yield_df = pd.read_csv(yield_filepath)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['bybit_ethcm_yield'] / 0.0175,
    mode='lines',
    name='Bybit ETHUSD',
    line=dict(color='red', width=2),
    opacity=0.7,
    legendrank=5
))

fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['okx_ethcm_yield'] / 0.0225,
    mode='lines',
    name='OKX ETH-USD-SWAP',
    line=dict(color='cyan', width=2),
    opacity=0.7,
    legendrank=6
))

fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['binance_ethcm_yield'] / 0.045,
    mode='lines',
    name='Binance ETHUSD_PERP',
    line=dict(color='orange', width=2),
    opacity=0.7,
    legendrank=4
))


fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['binance_ethusd_yield'] / 0.18,
    mode='lines',
    name='Binance ETH/USDT:USDT',
    line=dict(color='blue', width=2),
    opacity=0.7,
    legendrank=1
))


fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['bybit_ethusd_yield'] / 0.07,
    mode='lines',
    name='Bybit ETH/USDT:USDT',
    line=dict(color='green', width=2),
    opacity=0.7,
    legendrank=2
))


fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['okx_ethusd_yield'] / 0.09,
    mode='lines',
    name='OKX ETH-USDT-SWAP',
    line=dict(color='purple', width=2),
    opacity=0.7,
    legendrank=3
))


fig.update_layout(
    title='Historic ETH Perpetual Yields',
    yaxis_title='Yield (%)',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation='h',
        x=0.5,
        y=-0.2,
        xanchor='center',
        yanchor='top'
    ),
    width=800,
    height=600,
    margin=dict(l=40, r=40, t=80, b=40)
)

fig.update_xaxes(
    tickformat='%b %Y',
)

fig.show()

In [149]:
import pandas as pd
import plotly.graph_objects as go

yield_filepath = './results/test/yield.csv'
yield_df = pd.read_csv(yield_filepath)

total_yields = {
    'Binance ETH/USDT:USDT': (yield_df['binance_ethusd_yield']).sum(),  # / 0.36
    'Binance ETHUSD_PERP': (yield_df['binance_ethcm_yield']).sum(),  # / 0.09
    'Bybit ETH/USDT:USDT': (yield_df['bybit_ethusd_yield']).sum(),  # / 0.11
    'Bybit ETHUSD': (yield_df['bybit_ethcm_yield']).sum(),  # / 0.025
    'OKX ETH-USDT-SWAP': (yield_df['okx_ethusd_yield']).sum(),  # / 0.09
    'OKX ETH-USD-SWAP': (yield_df['okx_ethcm_yield']).sum(),  # / 0.0225
}

total_yields_df = pd.DataFrame(list(total_yields.items()), columns=['Asset', 'Cumulative Yield'])

fig = go.Figure()

fig.add_trace(go.Bar(
    x=total_yields_df['Asset'],
    y=total_yields_df['Cumulative Yield'],
    marker_color=['blue', 'orange', 'green', 'red', 'purple', 'cyan'],  
    opacity=1.0
))

fig.update_layout(
    title='Cumulative ETH Perpetual Yields',
    # xaxis_title='Asset',
    yaxis_title='Cumulative Yield (%)',
    template='plotly_white',
    hovermode='x unified',
    width=800,
    height=600,
    margin=dict(l=40, r=40, t=80, b=40)
)

fig.show()

In [154]:
import pandas as pd
import plotly.graph_objects as go

yield_filepath = './results/test/yield.csv'
yield_df = pd.read_csv(yield_filepath)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['binance_eth_yield'] / 0.18,
    mode='lines',
    name='Binance',
    line=dict(color='blue', width=2),
    opacity=0.7,
    legendrank=1
))

fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['bybit_eth_yield'] / 0.07,
    mode='lines',
    name='Bybit',
    line=dict(color='orange', width=2),
    opacity=0.7,
    legendrank=2
))

fig.add_trace(go.Scatter(
    x=yield_df['date'],
    y=yield_df['okx_eth_yield'] / 0.09,
    mode='lines',
    name='OKX',
    line=dict(color='green', width=2),
    opacity=0.7,
    legendrank=3
))

fig.update_layout(
    title='ETH Exchange Perpetual Yields',
    yaxis_title='Yield (%)',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation='h',
        x=0.5,
        y=-0.2,
        xanchor='center',
        yanchor='top'
    ),
    width=800,
    height=600,
    margin=dict(l=40, r=40, t=80, b=40)
)

fig.update_xaxes(
    tickformat='%b %Y',
)

fig.show()

In [132]:
import pandas as pd
import plotly.graph_objects as go

yield_filepath = './results/test/yield.csv'
yield_df = pd.read_csv(yield_filepath)

eth_columns = [
    'binance_ethusd_yield',
    'binance_ethcm_yield',
    'bybit_ethusd_yield',
    'bybit_ethcm_yield',
    'okx_ethusd_yield',
    'okx_ethcm_yield'
]

eth_yield_df = yield_df[eth_columns]


positive_counts = (eth_yield_df >= 0).sum().sum()
negative_counts = (eth_yield_df < 0).sum().sum()

fig = go.Figure()


fig.add_trace(go.Pie(
    values=[positive_counts, negative_counts], 
    labels=['Positive', 'Negative'],
    textinfo='percent',
    hole=0.7, 
    sort=False, 
    direction='counterclockwise', 
    marker=dict(colors=['green', 'red']),
    opacity=1.0
))

fig.update_layout(
    title_text='ETH Perpetual Yield Day Counts',
    showlegend=True,
    legend=dict(
        orientation='h',
        x=0.5,
        y=-0.2,
        xanchor='center',
        yanchor='top'
    ),
    height=300,
    width=300
)

fig.show()